# Score Engine — Final Accessibility Scoring (v1 + v2 + v3)

This notebook computes the **final Access4All accessibility score** for each Airbnb listing by combining three finalized layers:

- **v1 (Listing features):** elevator / private entrance / parking / service animals → `v1_score`, `v1_confidence`
- **v2 (OSM context):** nearby barriers and accessible POIs (200m) → `v2_score`, `v2_confidence`, `v2_hard_block`
- **v3 (Terrain):** Copernicus DEM slope metrics (200m) → `v3_score`, `v3_confidence`, `v3_hard_block`

The final score is computed using a **weighted harmonic mean** (non-compensatory), with a small terrain bonus, and a **hard-block override** that forces `final_score = 0` when severe barriers are detected.

Outputs:
- Writes the canonical table: `default.access4all_final_scores`
- Includes user-facing fields for filtering/constraints and short reason codes explaining limiting factors




### Compute v1 accessibility component: collapse to one row per listing and score core on-listing features with fixed weights


In [0]:
from pyspark.sql import functions as F

# Load v1
v1 = spark.table("default.access4all_airbnb_v1_9cities")

# Step 1: Conservative collapse to 1 row per property_id
# Using max() for booleans (True > False) to keep positive accessibility features
v1_collapsed = v1.groupBy('property_id').agg(
    F.max('has_elevator').alias('has_elevator'),
    F.max('has_private_entrance').alias('has_private_entrance'),
    F.max('has_parking_on_premises').alias('has_parking_on_premises'),
    F.max('assistance_animals_allowed_explicit').alias('assistance_animals_allowed_explicit'),
    F.first('city').alias('city')
)



# Step 2: Calculate v1_score using weighted sum
WEIGHT_ELEVATOR = 0.50
WEIGHT_PRIVATE_ENTRANCE = 0.20
WEIGHT_PARKING = 0.15
WEIGHT_SERVICE_ANIMALS = 0.15

v1_scored = v1_collapsed.withColumn(
    'v1_score',
    (
        F.when(F.col('has_elevator'), F.lit(WEIGHT_ELEVATOR)).otherwise(F.lit(0.0)) +
        F.when(F.col('has_private_entrance'), F.lit(WEIGHT_PRIVATE_ENTRANCE)).otherwise(F.lit(0.0)) +
        F.when(F.col('has_parking_on_premises'), F.lit(WEIGHT_PARKING)).otherwise(F.lit(0.0)) +
        F.when(F.col('assistance_animals_allowed_explicit'), F.lit(WEIGHT_SERVICE_ANIMALS)).otherwise(F.lit(0.0))
    )
).withColumn(
    'v1_confidence',
    F.lit(0.7)
)

# Step 3: Select final output columns (keeping constraint fields for webapp)
v1_final = v1_scored.select(
    'property_id',
    'v1_score',
    'v1_confidence',
    'city',
    'has_elevator',
    'has_private_entrance',
    'has_parking_on_premises',
    'assistance_animals_allowed_explicit'
)



### Compute v2 accessibility component from OSM context: penalties for step barriers, bonuses for nearby accessible POIs, plus conservative hard-block and confidence


In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window

# Load v2
v2 = spark.table("default.access4all_airbnb_v2_osm_9cities")

# Step 1: Conservative collapse to 1 row per property_id
# For counts: use max (if any duplicate reports higher count, keep it)
# For distances: use min (if any duplicate reports closer, keep it)
# For context: use mode or first (should be consistent)

v2_collapsed = v2.groupBy('property_id').agg(
    F.first('city').alias('city'),
    F.max('osm_poi_wheelchair_yes_count_200m').alias('osm_poi_wheelchair_yes_count_200m'),
    F.max('osm_poi_wheelchair_limited_count_200m').alias('osm_poi_wheelchair_limited_count_200m'),
    F.max('osm_poi_wheelchair_no_count_200m').alias('osm_poi_wheelchair_no_count_200m'),
    F.min('osm_poi_wheelchair_yes_nearest_distance_m').alias('osm_poi_wheelchair_yes_nearest_distance_m'),
    F.min('osm_poi_wheelchair_limited_nearest_distance_m').alias('osm_poi_wheelchair_limited_nearest_distance_m'),
    F.min('osm_poi_wheelchair_no_nearest_distance_m').alias('osm_poi_wheelchair_no_nearest_distance_m'),
    F.max('osm_steps_count_200m').alias('osm_steps_count_200m'),
    F.max('osm_steps_ramp_yes_count_200m').alias('osm_steps_ramp_yes_count_200m'),
    F.min('osm_steps_nearest_distance_m').alias('osm_steps_nearest_distance_m'),
    F.first('osm_access_context_200m').alias('osm_access_context_200m')
)


# Step 2: Calculate v2_score components

v2_scored = v2_collapsed

# Component 1: Step density penalty
v2_scored = v2_scored.withColumn(
    'step_density_penalty',
    F.when(F.col('osm_steps_count_200m') == 0, 0.00)
     .when(F.col('osm_steps_count_200m') <= 2, 0.05)
     .when(F.col('osm_steps_count_200m') <= 5, 0.12)
     .when(F.col('osm_steps_count_200m') <= 10, 0.20)
     .otherwise(0.30)
)

# Component 2: Step proximity penalty
v2_scored = v2_scored.withColumn(
    'step_proximity_penalty',
    F.when(F.col('osm_steps_nearest_distance_m') >= 150, 0.00)
     .when(F.col('osm_steps_nearest_distance_m') >= 100, 0.05)
     .when(F.col('osm_steps_nearest_distance_m') >= 50, 0.12)
     .otherwise(0.25)
)

# Component 3: Ramp mitigation
v2_scored = v2_scored.withColumn(
    'ramp_mitigation',
    F.when(
        F.col('osm_steps_count_200m') > 0,
        F.least(
            F.lit(0.15),
            (F.col('osm_steps_ramp_yes_count_200m') / F.col('osm_steps_count_200m')) * 0.20
        )
    ).otherwise(0.0)
)

# Total barrier penalty
v2_scored = v2_scored.withColumn(
    'barrier_penalty',
    F.least(
        F.lit(0.60),
        F.col('step_density_penalty') + F.col('step_proximity_penalty') - F.col('ramp_mitigation')
    )
)

# Component 4: Effective accessible POI count (including limited at 50% weight)
v2_scored = v2_scored.withColumn(
    'effective_accessible_poi_count',
    F.col('osm_poi_wheelchair_yes_count_200m') + (F.col('osm_poi_wheelchair_limited_count_200m') * 0.5)
)

# Component 5: POI count bonus
v2_scored = v2_scored.withColumn(
    'poi_count_bonus',
    F.when(F.col('effective_accessible_poi_count') == 0, 0.00)
     .when(F.col('effective_accessible_poi_count') <= 3, 0.10)
     .when(F.col('effective_accessible_poi_count') <= 10, 0.20)
     .when(F.col('effective_accessible_poi_count') <= 20, 0.30)
     .otherwise(0.35)
)

# Component 6: Distance modifier for POI bonus
v2_scored = v2_scored.withColumn(
    'poi_distance_modifier',
    F.when(F.col('osm_poi_wheelchair_yes_nearest_distance_m') < 50, 1.0)
     .when(F.col('osm_poi_wheelchair_yes_nearest_distance_m') < 100, 0.8)
     .when(F.col('osm_poi_wheelchair_yes_nearest_distance_m') < 150, 0.5)
     .when(F.col('osm_poi_wheelchair_yes_nearest_distance_m') < 200, 0.3)
     .otherwise(0.0)
)

# Total accessible POI bonus
v2_scored = v2_scored.withColumn(
    'accessible_poi_bonus',
    F.col('poi_count_bonus') * F.col('poi_distance_modifier')
)

# Component 7: Context adjustment
v2_scored = v2_scored.withColumn(
    'context_adjustment',
    F.when(F.col('osm_access_context_200m') == 'supportive', 0.10)
     .when(F.col('osm_access_context_200m') == 'barrier_dense', -0.10)
     .otherwise(0.00)
)

# Calculate raw v2_score
v2_scored = v2_scored.withColumn(
    'v2_score_raw',
    F.lit(0.50) - F.col('barrier_penalty') + F.col('accessible_poi_bonus') + F.col('context_adjustment')
)

# Clamp to [0, 1]
v2_scored = v2_scored.withColumn(
    'v2_score',
    F.when(F.col('v2_score_raw') < 0, 0.0)
     .when(F.col('v2_score_raw') > 1, 1.0)
     .otherwise(F.col('v2_score_raw'))
)

# Step 3: Hard block logic (very conservative)
v2_scored = v2_scored.withColumn(
    'v2_hard_block',
    (F.col('osm_steps_nearest_distance_m') < 30) &
    (F.col('osm_steps_count_200m') >= 15) &
    (F.col('osm_steps_ramp_yes_count_200m') == 0) &
    (F.col('osm_access_context_200m') == 'barrier_dense')
)

# Override score to 0 if hard blocked
v2_scored = v2_scored.withColumn(
    'v2_score',
    F.when(F.col('v2_hard_block'), 0.0).otherwise(F.col('v2_score'))
)

# Step 4: Calculate v2_confidence
v2_scored = v2_scored.withColumn(
    'total_poi_count',
    F.col('osm_poi_wheelchair_yes_count_200m') + 
    F.col('osm_poi_wheelchair_limited_count_200m') + 
    F.col('osm_poi_wheelchair_no_count_200m')
)

v2_scored = v2_scored.withColumn(
    'coverage_penalty',
    F.when(F.col('total_poi_count') == 0, 0.20)
     .when(F.col('total_poi_count') < 3, 0.10)
     .otherwise(0.00)
)

v2_scored = v2_scored.withColumn(
    'coverage_bonus',
    F.when(F.col('total_poi_count') > 30, 0.10).otherwise(0.00)
)

v2_scored = v2_scored.withColumn(
    'v2_confidence_raw',
    F.lit(0.70) - F.col('coverage_penalty') + F.col('coverage_bonus')
)

v2_scored = v2_scored.withColumn(
    'v2_confidence',
    F.when(F.col('v2_confidence_raw') < 0.50, 0.50)
     .when(F.col('v2_confidence_raw') > 0.85, 0.85)
     .otherwise(F.col('v2_confidence_raw'))
)

# Step 5: Select final output columns (keeping constraint fields for webapp)
v2_final = v2_scored.select(
    'property_id',
    'v2_score',
    'v2_confidence',
    'v2_hard_block',
    # Keep key fields for user constraints
    'osm_poi_wheelchair_yes_count_200m',
    'osm_poi_wheelchair_yes_nearest_distance_m',
    'osm_steps_count_200m',
    'osm_steps_nearest_distance_m',
    'osm_access_context_200m'
)



### Compute v3 accessibility component from terrain: slope-based penalties, conservative hard-blocks, and coverage-driven confidence


In [0]:
from pyspark.sql import functions as F

# Load v3 (already no duplicates, so no collapse needed)
v3 = spark.table("default.access4all_airbnb_v3_terrain_9cities")


# Step 1: Calculate v3_score components for properties WITH terrain data

v3_scored = v3

# Component 1: Median slope penalty (p50)
v3_scored = v3_scored.withColumn(
    'p50_penalty',
    F.when(F.col('slope_p50_pct_200m').isNull(), F.lit(None))
     .when(F.col('slope_p50_pct_200m') < 3, 0.00)
     .when(F.col('slope_p50_pct_200m') < 6, 0.05)
     .when(F.col('slope_p50_pct_200m') < 10, 0.15)
     .otherwise(0.30)
)

# Component 2: High percentile slope penalty (p90)
v3_scored = v3_scored.withColumn(
    'p90_penalty',
    F.when(F.col('slope_p90_pct_200m').isNull(), F.lit(None))
     .when(F.col('slope_p90_pct_200m') < 8, 0.00)
     .when(F.col('slope_p90_pct_200m') < 15, 0.10)
     .when(F.col('slope_p90_pct_200m') < 25, 0.20)
     .otherwise(0.35)
)

# Component 3: Steep area share penalty
v3_scored = v3_scored.withColumn(
    'area_penalty',
    F.when(F.col('share_area_slope_gt_8pct_200m').isNull(), F.lit(None))
     .when(F.col('share_area_slope_gt_8pct_200m') < 0.1, 0.00)
     .when(F.col('share_area_slope_gt_8pct_200m') < 0.3, 0.05)
     .when(F.col('share_area_slope_gt_8pct_200m') < 0.6, 0.15)
     .otherwise(0.25)
)

# Total terrain penalty (capped at 0.60)
v3_scored = v3_scored.withColumn(
    'terrain_penalty',
    F.when(
        F.col('slope_p50_pct_200m').isNull(),
        F.lit(None)
    ).otherwise(
        F.least(
            F.lit(0.60),
            F.col('p50_penalty') + F.col('p90_penalty') + F.col('area_penalty')
        )
    )
)

# Component 4: Context adjustment
v3_scored = v3_scored.withColumn(
    'context_adjustment',
    F.when(F.col('terrain_context_label_200m') == 'flat_supportive', 0.10)
     .when(F.col('terrain_context_label_200m') == 'steep_challenging', -0.10)
     .when(F.col('terrain_context_label_200m') == 'moderate', 0.00)
     .otherwise(0.00)  # unknown_coverage
)

# Calculate raw v3_score (for properties with data)
v3_scored = v3_scored.withColumn(
    'v3_score_raw',
    F.when(
        F.col('slope_p50_pct_200m').isNull(),
        F.lit(None)
    ).otherwise(
        F.lit(0.50) - F.col('terrain_penalty') + F.col('context_adjustment')
    )
)

# Clamp to [0, 1]
v3_scored = v3_scored.withColumn(
    'v3_score_clamped',
    F.when(F.col('v3_score_raw').isNull(), F.lit(None))
     .when(F.col('v3_score_raw') < 0, 0.0)
     .when(F.col('v3_score_raw') > 1, 1.0)
     .otherwise(F.col('v3_score_raw'))
)

# Step 2: SAFETY-FIRST NULL HANDLING
# If terrain metrics are NULL, set v3_score = 0.0 (fail-safe)
v3_scored = v3_scored.withColumn(
    'v3_score',
    F.when(F.col('slope_p50_pct_200m').isNull(), 0.0)
     .otherwise(F.col('v3_score_clamped'))
)

# Step 3: Hard block logic (very conservative)
v3_scored = v3_scored.withColumn(
    'v3_hard_block',
    F.when(F.col('slope_p50_pct_200m').isNull(), False)
     .otherwise(
         (F.col('slope_p50_pct_200m') > 15) &
         (F.col('slope_p90_pct_200m') > 30) &
         (F.col('share_area_slope_gt_8pct_200m') > 0.8) &
         (F.col('terrain_context_label_200m') == 'steep_challenging')
     )
)

# Override score to 0 if hard blocked
v3_scored = v3_scored.withColumn(
    'v3_score',
    F.when(F.col('v3_hard_block'), 0.0).otherwise(F.col('v3_score'))
)

# Step 4: Calculate v3_confidence
v3_scored = v3_scored.withColumn(
    'base_confidence',
    F.when(F.col('dem_coverage_label_200m') == 'good', 0.85)
     .when(F.col('dem_coverage_label_200m') == 'partial', 0.70)
     .otherwise(0.30)  # poor
)

v3_scored = v3_scored.withColumn(
    'void_penalty',
    F.when(F.col('dem_void_ratio_200m') > 0.3, 0.15)
     .when(F.col('dem_void_ratio_200m') > 0.15, 0.05)
     .otherwise(0.00)
)

# If metrics are NULL, force low confidence
v3_scored = v3_scored.withColumn(
    'v3_confidence_raw',
    F.when(
        F.col('slope_p50_pct_200m').isNull(),
        0.30  # Low confidence for unknown terrain
    ).otherwise(
        F.col('base_confidence') - F.col('void_penalty')
    )
)

v3_scored = v3_scored.withColumn(
    'v3_confidence',
    F.when(F.col('v3_confidence_raw') < 0.30, 0.30)
     .when(F.col('v3_confidence_raw') > 0.85, 0.85)
     .otherwise(F.col('v3_confidence_raw'))
)

# Step 5: Select final output columns (keeping constraint fields for webapp)
v3_final = v3_scored.select(
    'property_id',
    'v3_score',
    'v3_confidence',
    'v3_hard_block',
    # Keep key fields for user constraints
    'slope_p50_pct_200m',
    'slope_p90_pct_200m',
    'slope_max_pct_200m',
    'share_area_slope_gt_8pct_200m',
    'dem_coverage_label_200m',
    'terrain_context_label_200m'
)



### Combine v1 + v2 + v3 into the final accessibility score using a weighted harmonic mean, conservative hard-blocks, confidence, and user-facing reason codes


In [0]:
from pyspark.sql import functions as F


# Step 1: Join all three layers on property_id
# Use aliases to avoid column name conflicts
v1_aliased = v1_final.alias('v1')
v2_aliased = v2_final.alias('v2')
v3_aliased = v3_final.alias('v3')

combined = v1_aliased.join(v2_aliased, on='property_id', how='inner') \
                     .join(v3_aliased, on='property_id', how='inner')



# Step 2: Calculate weighted harmonic mean
# Weights: v1=0.20, v2=0.35, v3=0.45
W1 = 0.20
W2 = 0.35
W3 = 0.45

combined = combined.withColumn(
    'harmonic_denominator',
    (W1 / F.col('v1.v1_score')) + 
    (W2 / F.col('v2.v2_score')) + 
    (W3 / F.col('v3.v3_score'))
)

# Handle division by zero (if any score is 0, harmonic mean approaches 0)
combined = combined.withColumn(
    'weighted_harmonic_mean',
    F.when(
        (F.col('v1.v1_score') == 0) | (F.col('v2.v2_score') == 0) | (F.col('v3.v3_score') == 0),
        0.0
    ).otherwise(
        (W1 + W2 + W3) / F.col('harmonic_denominator')
    )
)

# Step 3: Calculate terrain bonus (only if v3_score >= 0.5)
combined = combined.withColumn(
    'terrain_bonus',
    F.when(
        F.col('v3.v3_score') >= 0.5,
        (F.col('v3.v3_score') - 0.5) * 0.15
    ).otherwise(0.0)
)

# Step 4: Calculate raw final score
combined = combined.withColumn(
    'final_score_raw',
    F.col('weighted_harmonic_mean') + F.col('terrain_bonus')
)

# Clamp to [0, 1]
combined = combined.withColumn(
    'final_score_clamped',
    F.when(F.col('final_score_raw') < 0, 0.0)
     .when(F.col('final_score_raw') > 1, 1.0)
     .otherwise(F.col('final_score_raw'))
)

# Step 5: Apply hard block override
combined = combined.withColumn(
    'any_hard_block',
    F.col('v2.v2_hard_block') | F.col('v3.v3_hard_block')
)

combined = combined.withColumn(
    'final_score',
    F.when(F.col('any_hard_block'), 0.0)
     .otherwise(F.col('final_score_clamped'))
)

# Step 6: Determine limiting layer (which score is lowest)
combined = combined.withColumn(
    'min_layer_score',
    F.least(F.col('v1.v1_score'), F.col('v2.v2_score'), F.col('v3.v3_score'))
)

combined = combined.withColumn(
    'limiting_layer',
    F.when(F.col('any_hard_block'), 
           F.when(F.col('v2.v2_hard_block'), 'v2_hard_block').otherwise('v3_hard_block'))
     .when(F.col('v1.v1_score') == F.col('min_layer_score'), 'v1')
     .when(F.col('v2.v2_score') == F.col('min_layer_score'), 'v2')
     .otherwise('v3')
)

# Step 7: Calculate final confidence (conservative: minimum of all three)
combined = combined.withColumn(
    'final_confidence',
    F.least(F.col('v1.v1_confidence'), F.col('v2.v2_confidence'), F.col('v3.v3_confidence'))
)

# Step 8: Generate reasons (top limiting factors)
combined = combined.withColumn(
    'reasons_array',
    F.array(
        # From v1
        F.when(~F.col('v1.has_elevator'), F.lit('No elevator')),
        F.when(~F.col('v1.has_private_entrance'), F.lit('No private entrance')),
        F.when(~F.col('v1.has_parking_on_premises'), F.lit('No on-site parking')),
        F.when(~F.col('v1.assistance_animals_allowed_explicit'), F.lit('Service animals not allowed')),
        
        # From v2
        F.when(F.col('v2.osm_access_context_200m') == 'barrier_dense', F.lit('Dense barriers nearby')),
        F.when(F.col('v2.osm_steps_nearest_distance_m') < 50, F.lit('Steps very close')),
        F.when(F.col('v2.osm_poi_wheelchair_yes_count_200m') == 0, F.lit('No accessible POIs nearby')),
        
        # From v3
        F.when(F.col('v3.terrain_context_label_200m') == 'steep_challenging', F.lit('Steep terrain')),
        F.when(F.col('v3.slope_p50_pct_200m').isNull(), F.lit('Terrain data unavailable')),
        F.when(F.col('v3.v3_hard_block'), F.lit('Extremely steep terrain')),
        
        # Hard blocks
        F.when(F.col('v2.v2_hard_block'), F.lit('Impassable barriers nearby'))
    )
)

# Remove nulls and limit to top 4 reasons
combined = combined.withColumn(
    'reasons',
    F.array_remove(F.col('reasons_array'), None)
)

combined = combined.withColumn(
    'reasons',
    F.slice(F.col('reasons'), 1, 4)
)

# Step 9: Select final output columns for webapp
final_output = combined.select(
    F.col('property_id'),
    F.col('final_score'),
    F.col('final_confidence'),
    F.col('limiting_layer'),
    F.col('reasons'),
    F.col('any_hard_block'),
    
    # City for filtering
    F.col('v1.city').alias('city'),
    
    # Layer scores (for debugging/analysis)
    F.col('v1.v1_score').alias('v1_score'),
    F.col('v2.v2_score').alias('v2_score'),
    F.col('v3.v3_score').alias('v3_score'),
    
    # Hard block flags
    F.col('v2.v2_hard_block').alias('v2_hard_block'),
    F.col('v3.v3_hard_block').alias('v3_hard_block'),
    
    # v1 constraint fields
    F.col('v1.has_elevator').alias('has_elevator'),
    F.col('v1.has_private_entrance').alias('has_private_entrance'),
    F.col('v1.has_parking_on_premises').alias('has_parking_on_premises'),
    F.col('v1.assistance_animals_allowed_explicit').alias('assistance_animals_allowed_explicit'),
    
    # v2 constraint fields
    F.col('v2.osm_poi_wheelchair_yes_count_200m').alias('osm_poi_wheelchair_yes_count_200m'),
    F.col('v2.osm_poi_wheelchair_yes_nearest_distance_m').alias('osm_poi_wheelchair_yes_nearest_distance_m'),
    F.col('v2.osm_steps_count_200m').alias('osm_steps_count_200m'),
    F.col('v2.osm_steps_nearest_distance_m').alias('osm_steps_nearest_distance_m'),
    F.col('v2.osm_access_context_200m').alias('osm_access_context_200m'),
    
    # v3 constraint fields
    F.col('v3.slope_p50_pct_200m').alias('slope_p50_pct_200m'),
    F.col('v3.slope_p90_pct_200m').alias('slope_p90_pct_200m'),
    F.col('v3.slope_max_pct_200m').alias('slope_max_pct_200m'),
    F.col('v3.share_area_slope_gt_8pct_200m').alias('share_area_slope_gt_8pct_200m'),
    F.col('v3.dem_coverage_label_200m').alias('dem_coverage_label_200m'),
    F.col('v3.terrain_context_label_200m').alias('terrain_context_label_200m')
)



### Persist the final accessibility scores as the canonical output table for downstream use


In [0]:
# Save to Delta table
final_output.write \
   .mode('overwrite') \
    .format('delta') \
    .saveAsTable('default.access4all_final_scores')